In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="dhOw3ym7s1Kivzengl6a")
project = rf.workspace().project("bolt-sgg5o")
model = project.version(4).model

# infer on a local image
print(model.predict("myimg.jpg", confidence=40, overlap=30).json())

# visualize your prediction
model.predict("myimg.jpg", confidence=40, overlap=30).save("prediction4.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 320.0, 'y': 320.5, 'width': 502.0, 'height': 273.0, 'confidence': 0.9469457864761353, 'class': 'bolt', 'class_id': 0, 'image_path': 'myimg.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '640', 'height': '640'}}


In [10]:
import cv2
import inference
import supervision as sv

annotator = sv.BoxAnnotator()

def on_prediction(predictions, image):
    labels = [p["class"] for p in predictions["predictions"]]
    detections = sv.Detections.from_roboflow(predictions)
    cv2.imshow(
        "Prediction",
        annotator.annotate(
            scene=image,
            detections=detections,
            labels=labels
        )
    ),
    cv2.waitKey(1)

inference.Stream(
    source="webcam", # or rtsp stream or camera id
    model="bolt-sgg5o/4", # from Universe
    output_channel_order="BGR",
    use_main_thread=True, # for opencv display
    on_prediction=on_prediction,
)

AttributeError: 'NoneType' object has no attribute 'encode'

In [1]:
!pip install inference

Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install inference

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall opencv-python

In [1]:
import os

# Set the Roboflow API key as an environment variable
os.environ["ROBOFLOW_API_KEY"] = "dhOw3ym7s1Kivzengl6a"


In [2]:
from inference.models.utils import get_roboflow_model

model = get_roboflow_model(model_id="bolt-sgg5o/4")

C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


In [3]:
from inference import InferencePipeline
from inference.core.interfaces.stream.sinks import render_boxes

pipeline = InferencePipeline.init(
    model_id="bolt-sgg5o/4",
    video_reference=0,
    on_prediction=render_boxes,
)
pipeline.join()

[01/11/24 11:41:35] ERROR    HTTP error encountered while requesting Roboflow API response: 404  ]8;id=290242;file://C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\inference\core\roboflow_api.py\roboflow_api.py]8;;\:]8;id=369474;file://C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\inference\core\roboflow_api.py#84\84]8;;\
                             Client Error: Not Found for url:                                                      
                             https://api.roboflow.com/vmware-712/bolt-sgg5o/active_learning?api_                   
                             key=dh***6a                                                                           

                    WARNING  Registration thread is already stopped.                             ]8;id=918244;file://C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\inference\core\active_learning\middlewares.py\middlewares.py]8;;\:]8;id=418026;file://C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\inference\core\active_learning\middlewares.py#225\225]8;;\

In [3]:
import cv2
from roboflow import Roboflow

# Initialize Roboflow
rf = Roboflow(api_key="dhOw3ym7s1Kivzengl6a")
project = rf.workspace().project("bolt-sgg5o")
model = project.version(4).model
print(model.predict("captured_image.jpg", confidence=40, overlap=30).json())

# visualize your prediction

model.predict("captured_image.jpg", confidence=40, overlap=30).save("c_pred.jpg")

loading Roboflow workspace...
loading Roboflow project...
{'predictions': [], 'image': {'width': '640', 'height': '480'}}


In [ ]:
#worked

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()

    if not ret:
        print("Error: Unable to capture frame from the webcam.")
        continue

    # Save the frame temporarily
    cv2.imwrite("temp_frame.jpg", frame)

    try:
        # Make predictions on the frame
        output = model.predict("temp_frame.jpg", confidence=40, overlap=30).json()
        print(output)

        # Visualize predictions on the frame
        for prediction in output['predictions']:
            x, y, width, height = prediction['x'], prediction['y'], prediction['width'], prediction['height']
            confidence = prediction['confidence']
            label = prediction['class']
            if confidence >= 0.5 :
            # Convert 'x', 'y', 'width', 'height' to bounding box coordinates
                xmin, ymin, xmax, ymax = int(x), int(y), int(x + width), int(y + height)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(frame, f"{label}: {confidence:.2f}", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    except Exception as e:
        print("No detection:", e)

    finally:
        # Display the frame
        cv2.imshow('detect', frame)

        # Release resources in a finally block
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
        elif cv2.waitKey(1) & 0xFF == ord('c'):
            cv2.imwrite('captured_image.jpg', frame)
            print('Image captured!')